Com a coluna conteudo resumida

In [2]:
import pandas as pd
df_final = pd.read_csv("../dados/df_final.csv")

Extrair informações importantes de texto

Limite de tokens: Gemini 1.5 Flash suporta até 1 milhão de tokens contextuais, mas é mais rápido e mais barato com textos curtos. Se o documento for muito longo, divida em trechos.

Prompt estruturado = melhor resultado: Use listas, marcadores e perguntas diretas.

Padronize o input: Remova cabeçalhos, assinaturas ou trechos irrelevantes.

Verifique consistência: LLMs podem gerar variações. Se precisar de formato JSON, peça explicitamente.

In [ ]:
import os
from dotenv import load_dotenv
import google.generativeai as genai

# Carrega variáveis do .env
load_dotenv()
API_KEY = os.getenv("GOOGLE_API_KEY")

# Configura o Gemini
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

def extrair_info(texto):
    prompt = f"""
    Extraia as seguintes informações do texto abaixo e retorne em formato JSON:

    {{
      "tipo_decisao": "...",
      "condenacao": true,
      "valores_multas": ["..."],
      "seguiu_nota_tecnica": true
    }}

    Texto:
    \"\"\"{texto}\"\"\"
    """
    response = model.generate_content(prompt)
    return response.text

In [5]:
df_final.head(1)

,id,ano_documento,descricao_tipo_documento,decisao_tribunal,setor_economico,data_processo,data_documento,diferenca_dias,assinaturas,id_unidade,mercado_relevante,descricao_especificacao,conteudo,resumo
0,abbdc450-7ff1-4990-9bcc-8ae4386592f0,2024,Voto Processo Administrativo,vazio,Fabricação de Peças e Acessórios para o Sistem...,2019-09-13,2024-05-23,1714,Outras,110000967.0,"pistões de motor, bronzinas, camisas, pinos, b...","- Desmembrado Bernd Brünig, Faustino Luigi Mi...",SEI/CADE - 1390607 - Voto Processo Administrat...,"['\nREPRESENTADOS(AS): BERND BRÜNIG, FAUSTINO ..."


In [13]:
df_final['resumo'].iloc[0]
extrair_info(df_final['resumo'].iloc[0])

'```json\n{\n  "tipo_decisao": "Condenação Parcial",\n  "condenacao": true,\n  "valores_multas": ["324.116,21"],\n  "seguiu_nota_tecnica": true\n}\n```\n'